In [ ]:
import numpy as np
import jax.numpy as jnp
import jax
import shapely
import shapely.geometry as shap_geom
import matplotlib.pyplot as plt

import toflux.src.utils as _utils
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.bc as _bc

import toflux.src.fe_struct as _fea_struct
import toflux.src.fe_fluid as _fea_fluid
import toflux.experiments.fsi.pressure_coupling as _fsi
import toflux.src.solver as _solv
import toflux.src.viz as _viz


_Ext = _utils.Extent
_FluidField = _fea_fluid.FluidField
_Direc = _utils.Direction

jax.config.update("jax_enable_x64", True)

In [ ]:
bbox = _mesher.BoundingBox(
  x=_Ext(min=0.0, max=2.0),
  y=_Ext(min=0.0, max=1.0),
)

struct_mesh = _mesher.GridMesh(
  bounding_box=bbox,
  nel=(100, 50),
  dofs_per_node=2,
  gauss_order=2,
)

fluid_mesh = _mesher.GridMesh(
  bounding_box=bbox,
  nel=(100, 50),
  dofs_per_node=3,
  gauss_order=2,
)

In [ ]:
struct_mat = _mat.StructuralMaterial(
  youngs_modulus=1.0e5,
  poissons_ratio=0.30,
)

deformation_model = _fea_struct.DeformationModel.SMALL

fluid_mat = _mat.FluidMaterial(
  mass_density=1.0,
  dynamic_viscosity=1.0,
)

In [ ]:
def btm_face_cond(node_coords):
  return jnp.all(node_coords[:, 1] <= 1.05 * struct_mesh.bounding_box.y.min)


btm_faces = _bc.identify_faces(struct_mesh, btm_face_cond)
n = len(btm_faces)
xv = (_Direc.X, jnp.zeros(n))
yv = (_Direc.Y, jnp.zeros(n))
btm_face_val = [xv, yv]


fixed_bc = _bc.DirichletBC(elem_faces=btm_faces, values=btm_face_val)
struct_bc = _bc.process_boundary_conditions([fixed_bc], struct_mesh)

In [ ]:
reynolds_num = 1.0
char_velocity = reynolds_num * fluid_mat.kinematic_viscosity / (bbox.ly)


def inlet_cond(node_coords):
  left = jnp.all(node_coords[:, 0] <= fluid_mesh.elem_size[0] / 2.0)
  return left


def outlet_cond(node_coords):
  right = jnp.all(node_coords[:, 0] >= bbox.lx - fluid_mesh.elem_size[0] / 2.0)
  return right


def btm_face_cond(node_coords):
  btm = jnp.all(node_coords[:, 1] <= fluid_mesh.elem_size[1] / 2.0)
  return btm


def top_face_cond(node_coords):
  top = jnp.all(node_coords[:, 1] >= bbox.ly - fluid_mesh.elem_size[1] / 2.0)
  return top


inlet_faces = _bc.identify_faces(fluid_mesh, inlet_cond)
n = len(inlet_faces)
u_profile = char_velocity * (1.0 - jnp.linspace(-1.0, 1.0, n) ** 2)
u_vel = (_FluidField.U_VEL, u_profile)
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
inlet_face_val = [u_vel, v_vel]

# outlet condition
outlet_faces = _bc.identify_faces(fluid_mesh, outlet_cond)
n = len(outlet_faces)
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
pres = (_FluidField.PRESSURE, jnp.zeros(n))
outlet_face_val = [v_vel, pres]

# top condition
top_faces = _bc.identify_faces(fluid_mesh, top_face_cond)
n = len(top_faces)
u_vel = (_FluidField.U_VEL, jnp.zeros(n))
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
top_face_val = [u_vel, v_vel]

# bottom condition
bottom_faces = _bc.identify_faces(fluid_mesh, btm_face_cond)
n = len(bottom_faces)
u_vel = (_FluidField.U_VEL, jnp.zeros(n))
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
bottom_face_val = [u_vel, v_vel]

inlet_bc = _bc.DirichletBC(elem_faces=inlet_faces, values=inlet_face_val, name="in")
outlet_bc = _bc.DirichletBC(elem_faces=outlet_faces, values=outlet_face_val, name="out")
top_bc = _bc.DirichletBC(elem_faces=top_faces, values=top_face_val, name="top")
bottom_bc = _bc.DirichletBC(elem_faces=bottom_faces, values=bottom_face_val, name="btm")

fluid_bc_list = [inlet_bc, outlet_bc, top_bc, bottom_bc]

fluid_bc = _bc.process_boundary_conditions(fluid_bc_list, fluid_mesh)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
_viz.plot_grid_mesh(fluid_mesh, ax=ax, colorbar=False)
_viz.plot_bc(fluid_bc_list, fluid_mesh, ax=ax)
plt.show()

In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.SCIPY_SPARSE,
    "rtol": 1.0e-3,
    "petsc_solver": {},
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-4},
}

flow_solver = _fea_fluid.FluidSolver(
  mesh=fluid_mesh, bc=fluid_bc, material=fluid_mat, solver_settings=solver_settings
)

In [ ]:
press_vel = jnp.zeros((fluid_mesh.num_dofs,))
press_vel = press_vel.at[fluid_bc["fixed_dofs"]].set(fluid_bc["dirichlet_values"])


In [ ]:
brep_hh = "toflux/brep/hungry_horse.geojson"

with open(brep_hh, "r") as f:
  geometry = shapely.from_geojson(f.read())


density = np.zeros((struct_mesh.num_elems,))

for e in range(struct_mesh.num_elems):
  if geometry.contains(shap_geom.Point(struct_mesh.elem_centers[e])):
    density[e] = 1.0

_viz.plot_grid_mesh(struct_mesh, density)

In [ ]:
min_inv_permeability = _mat.brinkman_bound(fluid_mat.dynamic_viscosity, 100.0 * bbox.lx)
max_inv_permeability = _mat.brinkman_bound(
  fluid_mat.dynamic_viscosity, 1.0e-2 * bbox.lx
)
init_inv_permeability = _mat.brinkman_bound(
  fluid_mat.dynamic_viscosity, 1.0e-1 * bbox.lx
)


inv_permeability_ext = _utils.Extent(min=min_inv_permeability, max=max_inv_permeability)
brink_inter_factor = _mat.calculate_interpolation_factor(
  inv_permeability_ext, init_inv_permeability, desired_mat_fraction=0.2
)
brinkman_penalty = _mat.compute_ramp_interpolation(
  prop=density,
  ramp_penalty=brink_inter_factor,
  prop_ext=inv_permeability_ext,
)

In [ ]:
press_vel = _solv.modified_newton_raphson_solve(flow_solver, press_vel, density * 1e9)

In [ ]:
u_velocity = press_vel[1 : fluid_mesh.num_dofs : 3].reshape(
  fluid_mesh.nelx + 1, fluid_mesh.nely + 1
)

v_velocity = press_vel[2 : fluid_mesh.num_dofs : 3].reshape(
  fluid_mesh.nelx + 1, fluid_mesh.nely + 1
)

pressure = press_vel[0 : fluid_mesh.num_dofs : 3].reshape(
  fluid_mesh.nelx + 1, fluid_mesh.nely + 1
)
plt.figure()
a = plt.imshow(
  np.flipud(u_velocity.reshape((fluid_mesh.nelx + 1, fluid_mesh.nely + 1)).T),
  cmap="jet",
)
plt.colorbar(a)
plt.title("U velocity field")

plt.figure()
a = plt.imshow(
  np.flipud(pressure.reshape((fluid_mesh.nelx + 1, fluid_mesh.nely + 1)).T), cmap="jet"
)
plt.colorbar(a)
plt.title("Pressure field")

In [ ]:
nx, ny = fluid_mesh.nelx + 1, fluid_mesh.nely + 1
x = fluid_mesh.nodes.coords[:, 0].reshape(nx, ny)
y = fluid_mesh.nodes.coords[:, 1].reshape(nx, ny)
u = press_vel[1 : fluid_mesh.num_dofs : 3].reshape(nx, ny)
v = press_vel[2 : fluid_mesh.num_dofs : 3].reshape(nx, ny)

X, Y = x.T, y.T
U, V = u.T, v.T

fig, ax = plt.subplots(figsize=(7, 3.5))

strm = ax.streamplot(
  X,
  Y,
  U,
  V,
  density=1.0,
  linewidth=1,
  arrowsize=1,
)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_aspect("equal")
ax.set_title("Velocity streamlines")
plt.tight_layout()
plt.show()

In [ ]:
disp = jnp.zeros((struct_mesh.num_dofs,)) + 1.0e-6
disp = disp.at[struct_bc["fixed_dofs"]].set(struct_bc["dirichlet_values"])

In [ ]:
lam, mu = struct_mat.lame_parameters
density = density + 1.0e-2
lam, mu = lam * density, mu * density

coupling_force = _fsi.compute_pressure_force(
  press_vel[0 : fluid_mesh.num_dofs : 3], struct_mesh, density
)
struct_bc["elem_forces"] = coupling_force
struct_solver = _fea_struct.FEA(
  mesh=struct_mesh,
  material=struct_mat,
  deformation_model=deformation_model,
  bc=struct_bc,
  solver_settings=solver_settings,
)
disp = _solv.newton_raphson_solve(struct_solver, disp, lam, mu)

In [ ]:
disp_elem = disp[struct_mesh.elem_dof_mat]
u_disp, v_disp = disp_elem[:, 0::2].mean(axis=1), disp_elem[:, 1::2].mean(axis=1)
disp_mag = np.sqrt(u_disp**2 + v_disp**2)

disp_mag = disp_mag * density

plt.figure()
a = plt.imshow(disp_mag.reshape((struct_mesh.nelx, struct_mesh.nely)).T,
  cmap="jet",
  origin="lower"
)
plt.colorbar(a)
plt.title("Displacement Magnitude")